In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%pwd

'/Users/sukhdeepkaur/MDS/block6/lab/aus_rain_cloud'

In [5]:

%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [6]:

# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [7]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 8.5 s, sys: 9.35 s, total: 17.9 s
Wall time: 13min 6s


In [8]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 20.9 s, sys: 3.95 s, total: 24.9 s
Wall time: 29.8 s


# Step 2 - Combining the csvs

In [78]:

%%time
%memit

import pandas as pd
files = glob.glob('figshareairline/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0,)
                .assign(model = os.path.basename(filenames).rsplit('daily')[0].replace('_', ''))
                for file in files)
              )

df.to_csv("figshareairline/combined_data.csv")

peak memory: 1692.81 MiB, increment: -69.48 MiB
CPU times: user 6min 8s, sys: 24.8 s, total: 6min 32s
Wall time: 6min 46s


machine(mac) - the CPU time : 6 min32 sec and wall time is 6 min 46s

# Step 3 - Loading and EDA in Python

## EDA 1 using pandas chunksize

In [85]:

%%time
%%memit
df = pd.read_csv("figshareairline/combined_data.csv")
print(df["model"].value_counts())

SAM0-UNICON    62513863
Name: model, dtype: int64
peak memory: 4356.98 MiB, increment: 1848.12 MiB
CPU times: user 1min 14s, sys: 34.6 s, total: 1min 48s
Wall time: 2min 9s


In [83]:

%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshareairline/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts)

SAM0-UNICON    62513863.0
dtype: float64
peak memory: 3048.96 MiB, increment: 416.56 MiB
CPU times: user 1min 5s, sys: 9.11 s, total: 1min 14s
Wall time: 1min 17s


In EDA1 - When chunksize is used, the memory usage is 416.56 mb, which is lower as compared to loading all the file in one time with the memory usage of 1848 mb.

## EDA 2 using specific columns

In [7]:
%%time
%%memit

df = pd.read_csv("figshareairline/combined_data.csv")
df.head(5)

peak memory: 2545.91 MiB, increment: 2401.05 MiB
CPU times: user 59.5 s, sys: 25.9 s, total: 1min 25s
Wall time: 1min 37s


In [8]:
%%time
%%memit
usecols =  ['time', 'rain (mm/day)', 'model']
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
df = pd.read_csv("figshareairline/combined_data.csv", usecols= usecols)
df.head(5)

peak memory: 3126.76 MiB, increment: 1336.37 MiB
CPU times: user 51 s, sys: 22.2 s, total: 1min 13s
Wall time: 1min 25s


In second EDA, I used few columns from the combined file, hence the memory usage is 1336 mb, which is lower as compared to loading the file with all the columns with the memory usage of 2401 mb.